

[lacentrale](https://www.lacentrale.fr/listing)



# Les voitures selon [lacentrale.fr](https://www.lacentrale.fr/listing)

<img src= ''>

L'objectif de ce projet est de collecter les données des voitures présents sur la page `https://www.lacentrale.fr/listing` du site lacentrale.fr.

Les données que nous collecterons seront les suivantes :
name
price
year
origin
registration_date
technical_inspection
first_hand
mileage
fuel_type
transmission
num_doors
num_seats
power
co2_emission
length
trunk_volume
critair_rating
combined_consumption

# Lacentrale

In [1]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


def getLacentralePage(url, page=None):
    if page != None:
        url = url + "?page=" + page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, features="lxml")

    return soup


def getDetailsCar(carCard):
    car = {
        "name": None,
        "price": None,
        "year": None,
        "origin": None,
        "registration_date": None,
        "technical_inspection": None,
        "first_hand": None,
        "mileage": None,
        "fuel_type": None,
        "transmission": None,
        "num_doors": None,
        "num_seats": None,
        "power": None,
        "co2_emission": None,
        "length": None,
        "trunk_volume": None,
        "critair_rating": None,
        "combined_consumption": None,
    }

    dom = etree.HTML(str(carCard))

    if (
        carCard.find(
            "div",
            class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3",
        )
        != None
    ):
        car["name"] = carCard.find(
            "div",
            class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3",
        ).text.strip()

    if carCard.find("span", class_="PriceInformation_classifiedPrice__b-Jae") != None:
        car["price"] = carCard.find(
            "span", class_="PriceInformation_classifiedPrice__b-Jae"
        ).text.strip()

    if len(dom.xpath('//*[@id="year"]/span[2]/span')) > 0:
        car["year"] = dom.xpath('//*[@id="year"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="origin"]/span[2]/span[1]')) > 0:
        car["origin"] = dom.xpath('//*[@id="origin"]/span[2]/span[1]')[0].text.strip()

    if len(dom.xpath('//*[@id="firstCirculationDate"]/span[2]/span')) > 0:
        car["registration_date"] = dom.xpath(
            '//*[@id="firstCirculationDate"]/span[2]/span'
        )[0].text.strip()

    if len(dom.xpath('//*[@id="technicalControl"]/span[2]/span[1]')) > 0:
        car["technical_inspection"] = dom.xpath(
            '//*[@id="technicalControl"]/span[2]/span[1]'
        )[0].text.strip()

    if len(dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')) > 0:
        car["first_hand"] = dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="mileage"]/span[2]/span[1]')) > 0:
        car["mileage"] = dom.xpath('//*[@id="mileage"]/span[2]/span[1]')[0].text.strip()

    if len(dom.xpath('//*[@id="energy"]/span[2]/span')) > 0:
        car["fuel_type"] = dom.xpath('//*[@id="energy"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="gearbox"]/span[2]/span')) > 0:
        car["transmission"] = dom.xpath('//*[@id="gearbox"]/span[2]/span')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="doors"]/span[2]/span')) > 0:
        car["num_doors"] = dom.xpath('//*[@id="doors"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="seats"]/span[2]/span')) > 0:
        car["num_seats"] = dom.xpath('//*[@id="seats"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')) > 0:
        car["power"] = dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="co2"]/span[2]/div/div/div')) > 0:
        car["co2_emission"] = dom.xpath('//*[@id="co2"]/span[2]/div/div/div')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="length"]/span[2]/span')) > 0:
        car["length"] = dom.xpath('//*[@id="length"]/span[2]/span')[0].text.strip()

    if (
        len(
            dom.xpath(
                '//*[@id="general-information"]/div/div[2]/div/div[2]/div[1]/section/ul/li[14]/div/button/span'
            )
        )
        > 0
    ):
        car["trunk_volume"] = dom.xpath(
            '//*[@id="general-information"]/div/div[2]/div/div[2]/div[1]/section/ul/li[14]/div/button/span'
        )[0].text.strip()

    if len(dom.xpath('//*[@id="critAir"]/span[2]/span[1]/span[1]')) > 0:
        car["critair_rating"] = dom.xpath('//*[@id="critAir"]/span[2]/span[1]/span[1]')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="consumption"]/span[2]')) > 0:
        car["combined_consumption"] = dom.xpath('//*[@id="consumption"]/span[2]')[
            0
        ].text.strip()

    return car


def getLacentraleCars(soup):
    # Cette fonction est à lancer sur la page avec les voitures à récupérer
    # on récupère toutes les voiture
    # chaque voiture est contenu dans une div avec une classe spécifique

    cars = soup.find_all("div", class_=["searchCardContainer", "boostVo__container"])

    data = []

    # Pour chaque voiture on récupère les détails qui nous intéressent
    # print(cars)
    for car in cars:
        a = (
            "https://www.lacentrale.fr/"
            + car.find("a", class_="Vehiculecard_Vehiculecard_vehiculeCard")["href"]
        )
        # print(a)
        page = getLacentralePage(a)
        data.append(getDetailsCar(page))

    return data

In [2]:
# URL of the site to scrape
url = "https://www.lacentrale.fr/listing"
max_page = 300
cars = []

# Retrieve car details from all pages
for current_page in tqdm(range(1, max_page + 1)):
    try:
        page = getLacentralePage(url, str(current_page))
        car_data = getLacentraleCars(page)
        cars.extend(car_data)
    except requests.RequestException as e:
        print(f"Failed to fetch page {current_page}: {e}")
        pass

# Create a DataFrame from the collected car data
df = pd.DataFrame(cars)
df

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [34:19<00:00,  6.87s/it]


,name,price,year,origin,registration_date,technical_inspection,first_hand,mileage,fuel_type,transmission,num_doors,num_seats,power,co2_emission,length,trunk_volume,critair_rating,combined_consumption,num_seats
0,DACIA DOKKER VAN,13 900 €,2016,France,11/05/2016,Requis,Non,83 979 km,Essence,Manuelle,5,None,7,D,"4,36 m",None,1,"5,4",2
1,PORSCHE MACAN phase 2,131 230 €,2021,France,22/06/2021,Non requis,Oui,25 000 km,Essence,Automatique,5,None,26,G,"4,69 m",None,1,"11,3",5
2,BMW SERIE 1 F40,36 930 €,2021,France,29/06/2021,Non requis,Oui,67 189 km,Diesel,Automatique,5,None,10,D,"4,32 m",None,2,"5,4",5
3,TOYOTA C-HR phase 2,33 530 €,2021,France,29/07/2021,Non requis,Non,23 391 km,Hybride essence électrique,Automatique,5,None,7,B,"4,40 m",None,1,None,5
4,BMW SERIE 1 E87 5 PORTES phase 2,13 490 €,2009,France,28/10/2009,Requis,Oui,80 073 km,Diesel,Manuelle,5,None,7,B,"4,24 m",None,3,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,MASERATI GRANCABRIO,95 630 €,2014,France,08/01/2015,Requis,Non,68 172 km,Essence,Automatique,2,None,38,G,"4,88 m",None,1,"9,8",4
5096,SUZUKI VITARA 4,28 710 €,2023,Importé,30/05/2023,Non requis,Non,12 km,Hybride essence électrique,Manuelle,5,None,7,C,None,None,Euro 1 et avant,None,NaN
5097,BMW SERIE 3 F31 TOURING,27 260 €,2020,Importé,05/03/2020,Non requis,Oui,75 667 km,Diesel,Manuelle,5,None,6,B,None,None,2,None,5
5098,NISSAN JUKE II,17 740 €,2022,France,27/06/2022,Non requis,Non,54 869 km,Essence,Manuelle,5,None,6,C,"4,21 m",None,1,"5,9",5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  5100 non-null   object
 1   price                 5100 non-null   object
 2   year                  5100 non-null   object
 3   origin                5100 non-null   object
 4   registration_date     5100 non-null   object
 5   technical_inspection  5100 non-null   object
 6   first_hand            5100 non-null   object
 7   mileage               5100 non-null   object
 8   fuel_type             5100 non-null   object
 9   transmission          5100 non-null   object
 10  num_doors             5060 non-null   object
 11  num_seats             0 non-null      object
 12  power                 5097 non-null   object
 13  co2_emission          4859 non-null   object
 14  length                4476 non-null   object
 15  trunk_volume          852 non-null    